<a href="https://colab.research.google.com/github/TurkuNLP/intro-to-nlp/blob/master/course_project_2023_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to HLT Project (Template)

- Student(s) Name(s): Omar Nasri, Konsta Nyman  
- Date: 22.04.2023
- Chosen Corpus: IMDB
- Contributions (if group project): 

Omar: Pre-prosessing, modeling, evaluation

Konsta: Hyperparameter tuning, results and summary

### Corpus information

- Description of the chosen corpus: Large Movie Review Dataset. This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.
- Paper(s) and other published materials related to the corpus: 

State-of-the-art leaderboard: https://paperswithcode.com/sota/sentiment-analysis-on-imdb

Related sentiment analysis paper: Sentiment Analysis for Movies Reviews Dataset Using Deep Learning Models Nehal Mohamed Ali, Marwa Mostafa Abd El Hamid and Aliaa Youssif

- State-of-the-art performance (best published results) on this corpus:

1
RoBERTa-large with LlamBERT
96.68
LlamBERT: Large-scale low-cost data annotation in NLP
2024

2
RoBERTa-large
96.54
LlamBERT: Large-scale low-cost data annotation in NLP
2024

3
XLNet
96.21
XLNet: Generalized Autoregressive Pretraining for Language Understanding
2019
Transformer

---

## 1. Setup

In [81]:
# Your code to install and import libraries etc. here
import datasets 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import torch 
import transformers
import evaluate

---

## 2. Data download and preprocessing

### 2.1. Download the corpus

In [82]:
# Your code to download the corpus here
dset = datasets.load_dataset('imdb')
#load the dataset from cache
display(dset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

### 2.2. Preprocessing

In [83]:
# Your code for any necessary preprocessing here
#Shuffle the dataset
dset = dset.shuffle(seed=42)
#Remove the unsupervised data part of the dataset as we dont need it for this task
del dset['unsupervised']

In [84]:
vectorizer = CountVectorizer(binary=True, max_features=25000)
text_list = [i['text'] for i in dset['train']]
vectorizer.fit(text_list)

def vectorize_example(examples, vectorizer): 
    vectorized = vectorizer.transform([examples["text"]])
    non_zero = vectorized.nonzero()[1]
    non_zero += 1
    return {'input_ids': non_zero}

# Conversion vocabulary 
idx2word = {v: k for k, v in vectorizer.vocabulary_.items()}

tokenized_data = dset.map(vectorize_example, num_proc=4, fn_kwargs={'vectorizer': vectorizer})

In [85]:
test_row = tokenized_data['train'][0]['input_ids']
convered_text = [idx2word[i] for i in test_row]
print(convered_text)

['above', 'action', 'actress', 'allah', 'americana', 'anders', 'area', 'argument', 'atari', 'beulah', 'bother', 'butch', 'bye', 'characterisation', 'clairvoyant', 'classical', 'compared', 'complicating', 'criminal', 'englishman', 'enjoyment', 'evaluated', 'factions', 'faraway', 'fur', 'goodbye', 'handbook', 'haven', 'howard', 'ifc', 'isaac', 'italian', 'judged', 'justice', 'languages', 'likeable', 'looming', 'maine', 'mayberry', 'moreau', 'noah', 'notable', 'onassis', 'oral', 'others', 'peoples', 'plotline', 'plotted', 'policeman', 'preferable', 'primed', 'quits', 'realm', 'relations', 'serio', 'similarity', 'simpler', 'spirited', 'spotlight', 'superficiality', 'suspected', 'thank', 'thatch', 'theater', 'thereafter', 'thick', 'things', 'thinker', 'tho', 'toad', 'took', 'violently', 'wayans', 'weak', 'weaken', 'weirdos', 'writings']


In [86]:
def collator(examples):
    batch = {"labels":torch.tensor(list(example["label"] for example in examples))}
    tensors = []
    max_len = max(len(example["input_ids"]) for example in examples)
    for example in examples:
        ids = torch.tensor(example["input_ids"])
        padded = torch.nn.functional.pad(ids, (0, max_len - ids.shape[0]))
        tensors.append(padded)
    batch["input_ids"] = torch.vstack(tensors)
    return batch

---

## 3. Machine learning model

### 3.1. Model training

In [87]:
# Your code to train the machine learning model on the training set and evaluate the performance on the validation set here

class MLPConfig(transformers.PretrainedConfig):
    pass
class MLP(transformers.PreTrainedModel):
    config_class=MLPConfig
    def __init__(self,config):
        super().__init__(config)
        self.vocab_size=config.vocab_size #embedding matrix row count
        self.embedding=torch.nn.Embedding(num_embeddings=self.vocab_size+1,embedding_dim=config.hidden_size,padding_idx=0)
        torch.nn.init.uniform_(self.embedding.weight.data,-0.001,0.001) 
        self.output=torch.nn.Linear(in_features=config.hidden_size,out_features=config.nlabels)

    def forward(self,input_ids,labels=None):
        embedded=self.embedding(input_ids)
        embedded_summed=torch.sum(embedded,dim=1)
        projected=torch.tanh(embedded_summed) 
        logits=self.output(projected)
        if labels is not None:
            loss=torch.nn.CrossEntropyLoss()
            return (loss(logits,labels),logits)
        else:
            return (logits,)

### 3.2 Hyperparameter optimization

In [88]:
# Your code for hyperparameter optimization here
learning_rates = [1e-5, 1e-4, 1e-3]
batch_sizes = [32, 64, 128]

def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels
    predictions = np.argmax(outputs, axis=-1) #pick the index of the "winning" label
    return accuracy.compute(predictions=predictions, references=labels)

accuracy = evaluate.load("accuracy")

mlp_config=MLPConfig(vocab_size=len(vectorizer.vocabulary_),hidden_size=20,nlabels=2)

best_learning_rate = None
best_batch_size = None
best_accuracy = 0
for lr in learning_rates:
    for batch_size in batch_sizes:
        print(f"Training with lr={lr} and batch_size={batch_size}")
        mlp=MLP(mlp_config)
        trainer_args = transformers.TrainingArguments(
            "mlp_checkpoints", #save checkpoints here
            evaluation_strategy="steps",
            logging_strategy="steps",
            eval_steps=500,
            logging_steps=500,
            learning_rate=lr, #learning rate of the gradient descent
            max_steps=20000,
            load_best_model_at_end=True,
            per_device_train_batch_size=batch_size,
        )
        early_stopping = transformers.EarlyStoppingCallback(5)
        trainer = transformers.Trainer(
            model=mlp,
            args=trainer_args,
            train_dataset=tokenized_data["train"].select(range(10000)),
            eval_dataset=tokenized_data["test"].select(range(1000)), #make a smaller subset to evaluate on
            compute_metrics=compute_accuracy,
            data_collator=collator,
            callbacks=[early_stopping]
        )
        trainer.train()
        eval_result = trainer.evaluate(tokenized_data["test"])
        if eval_result["eval_accuracy"] > best_accuracy:
            best_accuracy = eval_result["eval_accuracy"]
            best_learning_rate = lr
            best_batch_size = batch_size
            

Training with lr=1e-05 and batch_size=32


C:\Users\omarn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  2%|▎         | 500/20000 [00:03<01:49, 177.75it/s]

{'loss': 0.6772, 'grad_norm': 0.8402527570724487, 'learning_rate': 9.75e-06, 'epoch': 1.6}


                                                    
  2%|▎         | 500/20000 [00:03<01:49, 177.75it/s]

{'eval_loss': 0.6655793190002441, 'eval_accuracy': 0.792, 'eval_runtime': 0.1628, 'eval_samples_per_second': 6142.872, 'eval_steps_per_second': 767.859, 'epoch': 1.6}


...

                                                    
 15%|█▌        | 3000/20000 [00:50<04:46, 59.39it/s]

{'eval_loss': 0.6868419051170349, 'eval_accuracy': 0.837, 'eval_runtime': 0.1694, 'eval_samples_per_second': 5904.593, 'eval_steps_per_second': 738.074, 'epoch': 37.97}


 15%|█▌        | 3000/20000 [00:50<04:45, 59.59it/s]


{'train_runtime': 50.345, 'train_samples_per_second': 50849.101, 'train_steps_per_second': 397.259, 'train_loss': 0.025013207534948984, 'epoch': 37.97}


100%|██████████| 3125/3125 [00:04<00:00, 753.12it/s]


In [95]:
print(f"Best accuracy: {best_accuracy}")
print(f"Best learning rate: {best_learning_rate}")
print(f"Best batch size: {best_batch_size}")

Best accuracy: 0.87948
Best learning rate: 0.0001
Best batch size: 32


In [99]:
#Create a new model with the best hyperparameters

mlp_config=MLPConfig(vocab_size=len(vectorizer.vocabulary_),hidden_size=20,nlabels=2)
mlp=MLP(mlp_config)

trainer_args = transformers.TrainingArguments(
    "mlp_checkpoints", #save checkpoints here
    evaluation_strategy="steps",
    logging_strategy="steps",
    eval_steps=500,
    logging_steps=500,
    learning_rate=1e-4, #learning rate of the gradient descent
    max_steps=20000,
    load_best_model_at_end=True,
    per_device_train_batch_size=32
)

accuracy = evaluate.load("accuracy")

def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels
    predictions = np.argmax(outputs, axis=-1) #pick the index of the "winning" label
    return accuracy.compute(predictions=predictions, references=labels)

early_stopping = transformers.EarlyStoppingCallback(5)

trainer = transformers.Trainer(
    model=mlp,
    args=trainer_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"].select(range(5000)), #make a smaller subset to evaluate on
    compute_metrics=compute_accuracy,
    data_collator=collator,
    callbacks=[early_stopping]
)

trainer.train()

C:\Users\omarn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  2%|▎         | 500/20000 [00:03<02:02, 159.70it/s]

{'loss': 0.5513, 'grad_norm': 0.8755040764808655, 'learning_rate': 9.75e-05, 'epoch': 0.64}


                                                    
  3%|▎         | 518/20000 [00:04<05:53, 55.16it/s] 

{'eval_loss': 0.4453277289867401, 'eval_accuracy': 0.8516, 'eval_runtime': 0.8962, 'eval_samples_per_second': 5578.944, 'eval_steps_per_second': 697.368, 'epoch': 0.64}


  5%|▌         | 1000/20000 [00:07<01:58, 159.85it/s]

...

 28%|██▊       | 5500/20000 [00:42<01:36, 149.65it/s]

{'loss': 0.0994, 'grad_norm': 0.9777780175209045, 'learning_rate': 7.25e-05, 'epoch': 7.03}


                                                     
 28%|██▊       | 5500/20000 [00:43<01:54, 127.02it/s]

{'eval_loss': 0.30186718702316284, 'eval_accuracy': 0.8826, 'eval_runtime': 0.8137, 'eval_samples_per_second': 6144.438, 'eval_steps_per_second': 768.055, 'epoch': 7.03}
{'train_runtime': 43.3017, 'train_samples_per_second': 14780.019, 'train_steps_per_second': 461.876, 'train_loss': 0.21797148271040484, 'epoch': 7.03}


TrainOutput(global_step=5500, training_loss=0.21797148271040484, metrics={'train_runtime': 43.3017, 'train_samples_per_second': 14780.019, 'train_steps_per_second': 461.876, 'train_loss': 0.21797148271040484, 'epoch': 7.03})

### 3.3. Evaluation on test set

In [100]:
# Your code to evaluate the final model on the test set here
test_results = trainer.predict(tokenized_data["test"].select(range(5000)))
test_accuracy = compute_accuracy((test_results.predictions, test_results.label_ids))
print(f"Test accuracy: {test_accuracy}")

100%|██████████| 625/625 [00:00<00:00, 766.48it/s]

Test accuracy: {'accuracy': 0.8874}


In [101]:
#Convert the 10 first predictions to labels 
print("Predictions:", "\n", test_results.predictions[:10])
print("Binary predicted labels:", np.argmax(test_results.predictions[:10], axis=-1))


#Print first 10 true labels
true_labels = test_results.label_ids[:10]
print("True labels:", true_labels)


Predictions: 
 [[-1.650777    2.4864943 ]
 [ 0.36898008 -0.0657791 ]
 [ 0.26644838  0.04376237]
 [-1.0865515   1.7044837 ]
 [ 0.6988572  -0.5024215 ]
 [-0.9906143   1.6176448 ]
 [ 0.6038404  -0.3825643 ]
 [ 1.8174835  -1.8273368 ]
 [ 0.7308029  -0.5329372 ]
 [-0.6699647   1.1603235 ]]
Binary predicted labels: [1 0 0 1 0 1 0 0 0 1]
True labels: [1 1 0 1 0 1 1 0 0 1]


---

## 4. Results and summary

### 4.1 Corpus insights

The corpus "imdb" consists of movie reviews from IMDB: 25,000 positive and 25,000 negative reviews. Each entry consists of the review and the corresponding sentiment label. Only highly polarizing reviews are considered in this dataset - no neutral reviews are included. People have written movie reviews and given the movie a score from 1 to 10. Reviews with a score of <= 4 are labeled as negative and reviews with a score of >= 7 are labeled positive. No more than 30 reviews per movie are included.

### 4.2 Results

We got an evaluation accuracy of 88.02%. We performed hyperparameter tuning on a subset of the data and found the best learning rate (1e-4) and batch size (32), which got us to an accuracy of 88.74%. Since evaluation loss starts to rise early (and the training loss keeps decreasing), the model is most likely overfitting. 

### 4.3 Relation to state of the art

The state-of-the-art results of binary classifiers of the "imdb" dataset reach an accuracy of 96.68% with a RoBERTa-large with LlamBERT model. BERT is the standard state-of-the-art model in all NLP. BERT is a language model develped by Google so our accuracy with such a simple model can be viewed as a success. Nehal et al. got an accuracy of 86.74% in their paper using an MLP model.

---

## 5. Bonus Task (optional)

### 5.1. Annotating out-of-domain documents

(Briefly describe the chosen out-of-domain documents)

(Briefly describe the process of annotation)

### 5.2 Conversion into dataset

In [92]:
# Your code to convert the annotations into a dataset here

### 5.3. Model evaluation on out-of-domain test set

In [93]:
# Your code to evaluate the model on the out-of-domain test set here

### 5.4 Bonus task results

(Present the results of the evaluation on the out-of-domain test set)

### 5.5. Annotated data

In [94]:
# Include your annotated out-of-domain data here